In [5]:
import findspark
findspark.init("/usr/local/spark")

from pyspark.sql import SparkSession 
import pyspark.sql.functions as F 

from delta import *

builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .master("spark://namenode:7077")\

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [6]:
from pyspark.sql.types import *

review_schema = StructType([StructField("review_id", StringType(), False),
      StructField("user_id", StringType(), False),
      StructField("business_id", StringType(), False),
      StructField("stars", StringType(), False), 
      StructField("useful", IntegerType(), False),
      StructField("funny", IntegerType(), False),
      StructField("cool", IntegerType(), False),
      StructField("text", StringType(), False),
      StructField("date", StringType(), False),])
review_df = spark.read.csv("hdfs://namenode:9000/project_data/review.csv", sep = '|', header = False, schema = review_schema)
review_df.createOrReplaceTempView("reviews")


spark.sparkContext.setJobDescription('load review dataset')
business_schema = StructType([
    StructField("business_id", StringType(), False),
    StructField("name", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("stars", DoubleType(), True),
    StructField("review_count", IntegerType(), True),
    StructField("is_open", IntegerType(), True),
    StructField("attributes", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("hours", StringType(), True)
])
business_df = spark.read.csv("hdfs://namenode:9000/project_data/business.csv", sep = '|', header = False, schema = business_schema)
business_df.createOrReplaceTempView("business")

spark.sparkContext.setJobDescription('combine datasets')
reviews_with_category = spark.sql("SELECT r.review_id, r.user_id, r.business_id, r.stars, r.useful, r.funny, r.cool, r.text, r.date FROM reviews AS r LEFT JOIN business AS b ON b.business_id = r.business_id ")

spark.sparkContext.setJobDescription('filter reviews')
restaurant_reviews = reviews_with_category.where(F.col('categories').contains("Restaurants"))
review_df.show()

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|MmmrqbdupYBJq9jnl...|TjD2S66AWxlKsPSQB...|znK6tMeegKf9YnGut...|  5.0|     0|    0|   0|Great little spot...|2021-04-16 22:02:21|
|1_uWX0bBJNnZjPEcp...|Vtfxxq3nxdYHRiH6O...|UxwpCVLgPWCeaRyet...|  5.0|     0|    0|   0|James was super k...|2021-01-03 21:54:48|
|WVAEE47MnM3Sv2cvM...|ZDw_qN5Fy6PE0gdbV...|eVI64EQymywsvMLmD...|  5.0|     0|    0|   0|By far the best i...|2020-11-01 19:50:03|
|D5b6iooZZcJ8nG-9P...|UvMDlX2wV4Md9OwGW...|SPcPJfPgWzhjUDqVF...|  5.0|     0|    0|   0|I thought that th...|2021-03-24 05:53:49|
|uDxnT7zWFnxEDxp5e...|ir9ixBZPzBwWe9IZ_...|T5XzQ6YnVExvd0BOR...|  5.0|     0|    0|   0|As

In [8]:
fraction = 20 / review_df.count()

In [9]:
fraction

2.8611157206864388e-06

In [10]:
small_df = review_df.sample(fraction=fraction, seed=42)

In [11]:
small_df.show()

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|4WM6pP7lIin8-xyTb...|MDyb1LR5zsxkERwWx...|JpJogTQGjOQrA7yt1...|  5.0|     3|    0|   2|I moved my daught...|2018-08-09 20:55:33|
|ApYU3mkJPVqpsrDd5...|oSCobs0cPujMDavG_...|LdECsE8lJS7v5GTFT...|  5.0|     0|    0|   0|We are here.and l...|2018-09-12 00:09:57|
|wcNjtKfLSubxPp8kc...|Cex81esLlv8OtLjT-...|MZ9ucV1npggMOYY9f...|  5.0|     2|    0|   0|I recently purcha...|2019-09-01 03:55:28|
|JzQmAnl57t0zzAujo...|imjPUEXVN8rVv5MBh...|2bfExBV5NEmn79wUw...|  4.0|     2|    1|   3|"This is a fun ad...|2013-03-18 00:33:53|
|4pw-OuNdrzzZx1NDa...|oxO5avyhcsM7KyOuB...|-CbBGlrmddJsaruk6...|  2.0|     0|    0|   0|Th

In [12]:
small_df.count()

19

In [15]:
small_df.write.option("sep","|").mode("overwrite").csv("hdfs://namenode:9000/project_data/review_small_a.csv")

In [16]:
small_df2 = review_df.sample(fraction=fraction, seed=100)

In [17]:
small_df2.show()

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|wn1s1lPOuARSuMdo5...|Ozl9kIv9bA65DGHgc...|7jSm5z3n2JfDHsFAX...|  1.0|     1|    0|   1|Live around the c...|2015-08-15 20:24:27|
|nPFIpBocQmxtuFVv6...|saqaTwYnk6fY93sWR...|oeyfCPVjK9dv8pgLM...|  5.0|     0|    0|   0|It breaks our hea...|2020-09-14 18:06:15|
|lr1xBBahWP2ZOqGco...|GUkKB286tUKo-C-Uo...|rv_1S0yg-YtulZ6gG...|  5.0|     2|    0|   2|Great coffee shop...|2020-06-08 02:07:44|
|UQvcvXb6IZsXJxNhZ...|Efw1BpyljToAa-9mk...|P3EKMH2N1d5SbP1QI...|  5.0|     2|    0|   0|Does a great job ...|2019-10-10 02:56:50|
|GfbZu-2hg6WzG5b8v...|DDItoaNRmJJYrhJGz...|sJ7RwRN1Pd602KDXb...|  3.0|     1|    0|   1|Pa

In [18]:
small_df2.count()

25

In [19]:
small_df3 = small_df2.sample(fraction = 0.5)

In [20]:
small_df3.count()

14

In [21]:
small_df2.write.option("sep","|").mode("overwrite").csv("hdfs://namenode:9000/project_data/review_small_a.csv")

In [22]:
small_df = small_df.union(small_df3)

In [23]:
small_df.count()

33

In [24]:
small_df.write.option("sep","|").mode("overwrite").csv("hdfs://namenode:9000/project_data/review_small_b.csv")

In [22]:
delta_table_path = '/temp/reviews_analyzed_presentation'

# Check if the Delta table exists
if DeltaTable.isDeltaTable(spark, delta_table_path):
    print("Updating delta")
    # If the Delta table exists, load it into a DataFrame
    deltaTableAnalyzedReviews = DeltaTable.forPath(spark, delta_table_path)
    deltaTableAnalyzedReviews.alias('old') \
        .merge(
        small_df.alias('updates'),
        'old.review_id = updates.review_id'
  ) \
  .whenMatchedUpdate(set =
    {
      "review_id": "updates.review_id",
      "user_id": "updates.user_id",
      "business_id": "updates.business_id",
      "stars": "updates.stars",
      "useful": "updates.useful",
      "funny": "updates.funny",
      "cool": "updates.cool",
      "text": "updates.text",
      "date": "updates.date"
    }
  ) \
  .whenNotMatchedInsert(values =
    {
      "review_id": "updates.review_id",
      "user_id": "updates.user_id",
      "business_id": "updates.business_id",
      "stars": "updates.stars",
      "useful": "updates.useful",
      "funny": "updates.funny",
      "cool": "updates.cool",
      "text": "updates.text",
      "date": "updates.date"
    }
  ) \
  .execute()
else:
    # If the Delta table does not exist, create it
    print("Creating delta")
    deltaTableAnalyzedReviews = small_df3.write.format('delta').mode('overwrite').save(delta_table_path)
    deltaTableAnalyzedReviews = DeltaTable.forPath(spark, delta_table_path)
print("Finished!")

Updating delta


In [23]:
deltaTableAnalyzedReviews = DeltaTable.forPath(spark, delta_table_path)
df = deltaTableAnalyzedReviews.toDF()

# Use the DataFrame as desired
print(df.count())
df.show()

+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|           review_id|             user_id|         business_id|stars|useful|funny|cool|                text|               date|
+--------------------+--------------------+--------------------+-----+------+-----+----+--------------------+-------------------+
|-KOxrHhTSayEk01XH...|LyWYxyDpabqD8mgpr...|CrP6JWXBmf_HyMnZJ...|  4.0|     0|    0|   0|Good place to eat...|2013-02-20 17:34:12|
|0oCbH-K0tFa0Exbjl...|mlxzgOXyVl-7DryCd...|UakVMT3xrpbFB2pHd...|  5.0|     0|    0|   0|Well I finally ha...|2018-06-01 22:47:56|
|2ecMImprVq9UhB6Pe...|Oy3yB9lYVzcBV8OYb...|iUZEGx29miZObLd6_...|  5.0|     1|    1|   1|Wow! First off, I...|2009-11-20 16:54:17|
|4F0rrBlJ2GIbWcNNW...|xZLxP_yI5V2XyUET8...|UakVMT3xrpbFB2pHd...|  4.0|     0|    0|   0|Great breakfast b...|2018-12-24 18:49:00|
|4KjHo2y4YughHrdd4...|v8iT4FtpGv28yihHu...|KEcxJdAXJq1TDMrXZ...|  5.0|     3|    0|   1|Th

In [3]:
from pyspark.sql.types import *

review_schema = StructType([StructField("review_id", StringType(), False),
      StructField("user_id", StringType(), False),
      StructField("business_id", StringType(), False),
      StructField("stars", StringType(), False), 
      StructField("useful", IntegerType(), False),
      StructField("funny", IntegerType(), False),
      StructField("cool", IntegerType(), False),
      StructField("text", StringType(), False),
      StructField("date", StringType(), False),])
review_dfa = spark.read.csv("hdfs://namenode:9000/project_data/review_small_a.csv", sep = '|', header = False, schema = review_schema)


In [4]:
review_dfa.show()

+--------------------+-------+-----------+-----+------+-----+----+----+----+
|           review_id|user_id|business_id|stars|useful|funny|cool|text|date|
+--------------------+-------+-----------+-----+------+-----+----+----+----+
|2ecMImprVq9UhB6Pe...|   null|       null| null|  null| null|null|null|null|
|eUnl4eI0mEQcjdyee...|   null|       null| null|  null| null|null|null|null|
|LAlPaYMAWx0ZUNnDY...|   null|       null| null|  null| null|null|null|null|
|BXVm_qLhx2h0F1Cf-...|   null|       null| null|  null| null|null|null|null|
|0oCbH-K0tFa0Exbjl...|   null|       null| null|  null| null|null|null|null|
|ezwqc_Dj0FbnbJKjQ...|   null|       null| null|  null| null|null|null|null|
|uj7WMG5LTeGUt0Se3...|   null|       null| null|  null| null|null|null|null|
|cYTp9j-Rbmpij11dG...|   null|       null| null|  null| null|null|null|null|
|whUKoqf3hpklaiWmt...|   null|       null| null|  null| null|null|null|null|
|mUxl7h0ffYJHUhC9H...|   null|       null| null|  null| null|null|null|null|